# Benchmarking: QML vs Classical ML

When does quantum provide advantages?

**Time:** 20-25 minutes  
**Level:** Intermediate

In [ ]:
from quantum_debugger.benchmarks import (
    benchmark_qnn,
    compare_with_classical
)
from quantum_debugger.qml.qnn import QuantumNeuralNetwork
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import time

## Experiment 1: Small Dataset

In [ ]:
# Generate small dataset
X_small, y_small = make_classification(
    n_samples=100, n_features=4, n_informative=4,
    n_redundant=0, n_classes=2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small, test_size=0.3, random_state=42
)

print(f"Dataset: {len(X_train)} train, {len(X_test)} test samples")

In [ ]:
# Quantum ML
qnn = QuantumNeuralNetwork(n_qubits=4)
qnn.compile(optimizer='adam', loss='mse')

start = time.time()
qnn.fit(X_train, y_train, epochs=20, verbose=0)
qnn_time = time.time() - start

qnn_preds = (qnn.predict(X_test) > 0.5).astype(int).flatten()
qnn_acc = np.mean(qnn_preds == y_test)

print(f"QNN: {qnn_time:.2f}s, Accuracy: {qnn_acc:.3f}")

In [ ]:
# Classical ML
mlp = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=100, random_state=42)

start = time.time()
mlp.fit(X_train, y_train)
mlp_time = time.time() - start

mlp_acc = mlp.score(X_test, y_test)

print(f"Classical: {mlp_time:.2f}s, Accuracy: {mlp_acc:.3f}")

## Experiment 2: Automated Comparison

In [ ]:
# Use built-in comparison
comparison = compare_with_classical(
    n_qubits=4,
    dataset_size=100
)

print("\nComparison Results:")
print(f"QNN Accuracy: {comparison['qnn']['accuracy']:.3f}")
print(f"Classical Accuracy: {comparison['classical']['accuracy']:.3f}")
print(f"Speedup: {comparison['speedup']:.2f}x")
print(f"Quantum Advantage: {comparison['quantum_advantage']}")

## Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Time comparison
ax1.bar(['QNN', 'Classical'], [qnn_time, mlp_time])
ax1.set_ylabel('Time (s)')
ax1.set_title('Training Time')
ax1.grid(True, alpha=0.3, axis='y')

# Accuracy comparison
ax2.bar(['QNN', 'Classical'], [qnn_acc, mlp_acc])
ax2.set_ylabel('Accuracy')
ax2.set_title('Test Accuracy')
ax2.set_ylim([0, 1])
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## When to Use Quantum ML

### Quantum Advantages:
- Small datasets (< 1000 samples)
- High-dimensional features
- Quantum-generated data
- Exploratory research

### Classical Better For:
- Large datasets (> 10,000 samples)
- Production systems (stability)
- Well-understood problems
- Speed-critical applications

### Hybrid Approach:
Use quantum for feature extraction, classical for final prediction.